<a href="https://colab.research.google.com/github/apatel67/Assignment-2-Keras-Multi-class-Sentimental-Analysis/blob/master/Assignment_2_1116561(Keras).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Migrate tensor flow 1 code to tensor flow 2 
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#library for reading the dataset
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)

from sklearn.model_selection import train_test_split

import numpy as np

import nltk
import random
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
 # nltk.download('movie_reviews')

import csv
import urllib.request as urllib2

import matplotlib.pyplot as plt

from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer 

from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from keras.utils import to_categorical

import keras
from keras.datasets import mnist
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K
from keras.layers import SpatialDropout1D
from keras.layers.convolutional import Conv1D, MaxPooling1D

import time

Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Using TensorFlow backend.


In [2]:
# Here URL is given using which we import the data given in the Rotten tomatoes movie review
url = 'https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv'
response = urllib2.urlopen(url)

# pandas is used for reading the data  and converting the data into dataframe
df = pd.read_csv(response,delimiter='\t',encoding='utf-8')
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [3]:
df = df.sample(frac=1).reset_index(drop=True) #A new index label has been generated.
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,90113,4691,'ve yet to find an actual Vietnam War combat m...,2
1,60668,3065,perspicacious,2
2,130361,7022,feels all too familiar,1
3,22065,989,"and , for that matter , Shrek -RRB-",2
4,1821,67,the skin,2


In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(df ['Phrase'], df ['Sentiment'], test_size=0.3, random_state=1010)
documents=[]
X_train = np.array(X_train.values.tolist())
Y_train = np.array(Y_train.values.tolist())
for i in range(len(X_train)):
  documents.append([list(word_tokenize(X_train[i])), Y_train[i]]) 

X_test = np.array(X_test.values.tolist())
Y_test = np.array(Y_test.values.tolist())
for i in range(len(X_test)):
  documents.append([list(word_tokenize(X_test[i])), Y_test[i]]) 

documents[0]

[['trove'], 2]

In [0]:
porter = PorterStemmer() 
lancaster=LancasterStemmer() 
wordnet_lemmatizer = WordNetLemmatizer() 
stopwords_en = stopwords.words("english") 
punctuations="?:!.,;'\"-()"
#parameters to adjust to see the impact on outcome 
remove_stopwords = True
useStemming = False
useLemma = False
removePuncs = True

In [6]:
#Here we are trying to first save the review label for each document and then giving it a plcaeholder for new temporary review,
#After that we are trying to check each word in the documents are either stopwords or punctuations if there are then remove it 
#and jump to stemming using lancaster stemming. At last add that normalized word to new temporary review and updating it with new review  
for l in range(len(documents)):                    
  reviewlabel = documents[l][1]                          
  newReview = []                                   
  for w in documents[l][0]:                        
    newWord = w                                    
    if removePuncs and (w in punctuations):   
      continue                                     
    if remove_stopwords and (w in stopwords_en):        
      continue                                     
    if useStemming:
          
      newWord = lancaster.stem(newWord)  
    if useLemma: 
      newWord = wordnet_lemmatizer.lemmatize(newWord) 
    newReview.append(newWord)                      
  documents[l] = (newReview, reviewlabel)              
  documents[l] = (' '.join(newReview), reviewlabel) 

print(documents[0])

('trove', 2)


In [7]:
#Extracting text and sentiment from the dataset
df = pd.DataFrame(documents, columns=['text', 'sentiment']) 
df.head() 

,text,sentiment
0,trove,2
1,'s delivery matters,2
2,Lee -RRB-,2
3,rigid Blair Witch-style commitment mockumentar...,2
4,Philadelphia,2


In [0]:
#Again test & train as we did for phrase & sentiment
X_train, X_test, Y_train, Y_test = train_test_split(df['text'],  df['sentiment'], test_size=0.3, random_state=1010)

In [0]:
vectorizer = TfidfVectorizer(max_features = 2500)#, # ngram_range=(1, 1)) 
X = vectorizer.fit_transform(df["text"]) 
Y = df['sentiment'] 
 
X_train = vectorizer.transform(X_train).toarray()
Y_train = Y_train 
X_test = vectorizer.transform(X_test).toarray()
Y_test = Y_test

In [10]:
Y_test

90121     3
148474    3
144837    2
14154     0
113333    1
         ..
38492     2
139402    2
118392    1
19492     1
54104     2
Name: sentiment, Length: 46818, dtype: int64

In [0]:
batch_size = 64
num_classes = 10
epochs = 20

In [12]:
X_train.shape

(109242, 2500)

In [0]:
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes)

In [14]:
Y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [0]:
def recall_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true*y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0 ,1)))
  recall = true_positives / (possible_positives + K.epsilon())
  return recall

def precision_m(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true*y_pred, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0 ,1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  return precision

def f1_m(y_true, y_pred):
  precision = precision_m(y_true, y_pred)
  recall = recall_m(y_true, y_pred)
  return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [16]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3,activation='relu',input_shape=(2500,1)))
model.add(Conv1D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(rate = 0.5))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [17]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy',f1_m,precision_m,recall_m])

In [0]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [0]:
model.fit(X_train, Y_train,
          batch_size=64,
          epochs=20)
# _, accuracy = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
score = model.evaluate(X_test, Y_test, verbose=0)
print('loss of the model:', score[0])
print('Accuracy of the model:', score[1])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/20





109242/109242 [==============================] - 111s 1ms/step - loss: 1.1475 - acc: 0.5525 - f1_m: 0.4787 - precision_m: 0.5969 - recall_m: 0.4081
Epoch 2/20
109242/109242 [==============================] - 98s 894us/step - loss: 1.0482 - acc: 0.5922 - f1_m: 0.5516 - precision_m: 0.6544 - recall_m: 0.4785
Epoch 3/20
109242/109242 [==============================] - 98s 893us/step - loss: 1.0222 - acc: 0.6038 - f1_m: 0.5680 - precision_m: 0.6630 - recall_m: 0.4984
Epoch 4/20
109242/109242 [==============================] - 98s 895us/step - loss: 1.0086 - acc: 0.6110 - f1_m: 0.5781 - precision_m: 0.6702 - recall_m: 0.5097
Epoch 5/20
109242/109242 [==============================] - 98s 893us/step - loss: 1.0006 - acc: 0.6165 - f1_m: 0.5836 - precision_m: 0.6731 - recall_m: 0.5164
Epoch 6/20
109242/109242 [==============================] - 97s 892us/step - loss: 0.9944 - acc: 0.